In [1]:
import math
import os
import shutil
import time
import numpy as np
from datetime import datetime
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
import torchvision.models as models
from HSI_class import HSI
import createSample as CS
import augmentation as aug

import simsiam.loader
import simsiam.builder

start_time = time.time()

# Check if GPU is available
print("GPU Available:", torch.cuda.is_available())

# If available, print the GPU name
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    
sample_per_class = 50
num_per_category_augment_1 = 100
num_per_category_augment_2 = 100
epochs = 200

batch_size = 20
test_size = 0.5

random = 0


GPU Available: True
GPU Name: NVIDIA GeForce GTX 1650


In [2]:
dataset_path = r"C:\Users\Asus TUF\Documents\code\TA\Hyperspectral oil spill detection datasets"

dataset = []

i = 0
for filename in os.listdir(dataset_path):
    if i > 0:
        break
    file_path = os.path.join(dataset_path, filename)
    if os.path.isfile(file_path):  # Check if it's a file
        print(f"Processing file: {file_path}")
        hsi = HSI(file_path)
        dataset.append(hsi)
    i += 1

hsi_ = dataset[0]
patch_size = 9
sample_per_class = sample_per_class

indices_0 = []
indices_1 = []

print(f"random: {random}")

if random:
    print("generating random sample")
    selected_patch_0, selected_patch_1, indices_0, indices_1 = CS.createSample(hsi_, patch_size, sample_per_class)
else:
    print("using generated indices")
    indices_0 = [(np.int64(526), np.int64(187)), (np.int64(537), np.int64(71)), (np.int64(496), np.int64(222)), (np.int64(1200), np.int64(102)), (np.int64(1178), np.int64(413))]
    indices_1 = [(np.int64(174), np.int64(66)), (np.int64(382), np.int64(580)), (np.int64(1202), np.int64(171)), (np.int64(469), np.int64(254)), (np.int64(267), np.int64(228))]

    selected_patch_0, selected_patch_1 = CS.getSample(hsi_, patch_size, sample_per_class, indices_0, indices_1)


i =0
half_patch = patch_size // 2
# print(hsi_.img[indices_0[i][0]][indices_0[i][1]])
# print(selected_patch_0[i][half_patch][half_patch])

# print(hsi_.img[indices_1[i][0]][indices_1[i][1]])
# print(selected_patch_1[i][half_patch][half_patch])
# i =4
# half_patch = patch_size // 2
# print(hsi_.img[indices_0[i][0]][indices_0[i][1]])
# print(selected_patch_0[i][half_patch][half_patch])

# print(hsi_.img[indices_1[i][0]][indices_1[i][1]])
# print(selected_patch_1[i][half_patch][half_patch])

indices = indices_0 +  indices_1

# Concatenating along axis 0
x_train = np.concatenate((selected_patch_0, selected_patch_1), )

y_train = np.array([])

gt = hsi_.gt
for indice in indices:
    # print(gt[indice[0]][indice[1]])
    y_train = np.append(y_train, gt[indice[0]][indice[1]])

count = np.count_nonzero(y_train == 0)  # Count elements equal to 0
print(f'number of element equal 0 {count}')

count = np.count_nonzero(y_train == 1)  # Count elements equal to 1
print(f'number of element equal 1 {count}')



# Print shape to verify
print(f"x_train shape: {x_train.shape}")  # Expected output: (10, 9, 9, 224)
print(f"y_train shape: {y_train.shape}") 




Processing file: C:\Users\Asus TUF\Documents\code\TA\Hyperspectral oil spill detection datasets\GM01.mat
random: 0
using generated indices
hsi shape
(1243, 684, 224)
indices 0 used: [(np.int64(526), np.int64(187)), (np.int64(537), np.int64(71)), (np.int64(496), np.int64(222)), (np.int64(1200), np.int64(102)), (np.int64(1178), np.int64(413))]
indices 1 used: [(np.int64(174), np.int64(66)), (np.int64(382), np.int64(580)), (np.int64(1202), np.int64(171)), (np.int64(469), np.int64(254)), (np.int64(267), np.int64(228))]
number of element equal 0 5
number of element equal 1 5
x_train shape: (10, 9, 9, 224)
y_train shape: (10,)


In [3]:
n_category = 2
band_size = 224
num_per_category_augment_1 = num_per_category_augment_1
num_per_category_augment_2 = num_per_category_augment_2

data_augment1, label_augment1 = aug.Augment_data(x_train, y_train, n_category, patch_size, band_size, num_per_category_augment_1)

data_augment2, label_augment2 = aug.Augment_data2(x_train, y_train, n_category, patch_size, band_size, num_per_category_augment_2)

print(f"hasil augmentasi 1 shape: {data_augment1.shape}")
print(f"label augmentai 1 shape: {label_augment1.shape}")

print(f"hasil augmentasi 2 shape: {data_augment2.shape}")
print(f"label augmentasi 2 shape: {label_augment2.shape}")

print(label_augment1)
print(label_augment2)

# # Count occurrences of each unique element
# counts1 = np.bincount(label_augment1)

# # Print results
# for i, count in enumerate(counts1):
#     print(f"Element {i} occurs {count} times.")

# counts2 = np.bincount(label_augment2)

# # Print results
# for i, count in enumerate(counts2):
#     print(f"Element {i} occurs {count} times.")

# print(label_augment1[3])

data_augment = np.concatenate((data_augment1, data_augment2))
label_augment = np.concatenate((label_augment1, label_augment2))

print(f"hasil augmentasi gabungan untuk training: {data_augment.shape}")
print(f"label augmentasi gabungan: {label_augment.shape}")

# print(label_augment)

# Count occurrences of each unique element
counts = np.bincount(label_augment)

# Print results
for i, count in enumerate(counts):
    print(f"Element {i} occurs {count} times.")

hasil augmentasi 1 shape: (200, 9, 9, 224)
label augmentai 1 shape: (200,)
hasil augmentasi 2 shape: (200, 9, 9, 224)
label augmentasi 2 shape: (200,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
hasil augmentasi gabungan untuk tra

In [4]:
# import torch
# import torch.nn as nn
import torch.nn.functional as F

class ConvTo1D(nn.Module):
    def __init__(self):
        super(ConvTo1D, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=224, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))  # (batch_size, 128, 1, 1)
        self.flatten = nn.Flatten()               # (batch_size, 128)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = self.flatten(x)
        return x

In [5]:
print(data_augment.shape)
print(label_augment.shape)

x_data = data_augment 
y_labels = label_augment

x_data = torch.tensor(x_data)
x_data = x_data.to(torch.float32)
x_data = x_data.permute(0, 3, 1, 2)
print(f"X_train shape: {x_data.shape}")

feature_extractor = ConvTo1D()
with torch.no_grad():  # No gradient needed for feature extraction
    features = feature_extractor(x_data)  # (100, 128)
    features_np = features.numpy() 

    

print(features_np.shape)

(400, 9, 9, 224)
(400,)
X_train shape: torch.Size([400, 224, 9, 9])
(400, 128)


In [6]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

X = features_np
y = y_labels
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=42)

print(y_train)
print(y_val)

[1 0 0 0 0 1 0 0 1 1 0 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 0 0 1 0 0 0 1 1 0
 1 1 1 1 1 0 1 0 0 1 1 0 0 1 0 1 0 0 1 0 0 0 1 1 0 0 0 1 1 1 0 0 0 0 0 0 1
 1 0 1 0 0 1 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 1 0 1 1 0 0 1 0
 1 0 1 1 0 0 1 0 0 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0
 0 0 1 0 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 0 0 1 0 0 0 1 1 0 1 0 1 0 1 1 1 1
 1 0 1 0 1 0 1 1 0 1 0 1 0 1 1]
[0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 1 0 1 0 0 1 0 0 1 1 0 0 1 1 1 0 0 0 1 1
 1 1 1 1 0 0 0 0 1 1 1 0 1 0 1 1 0 1 0 0 1 0 1 1 0 0 0 1 0 1 0 1 0 1 1 0 0
 1 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 1 1 1 1 0 0 0 1 0 0 1 0 0 1 1 1 1
 1 0 1 0 0 0 0 1 0 1 1 0 1 0 1 1 1 1 0 1 1 0 1 1 1 0 0 0 1 0 1 0 1 1 0 0 1
 1 1 0 1 0 1 0 1 1 0 1 1 1 0 0 1 1 0 1 1 1 0 0 0 1 0 1 1 0 0 0 0 0 1 1 1 0
 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1]


In [7]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# --- Train SVM with probability enabled ---
pipeline = make_pipeline(StandardScaler(), SVC(kernel='linear', probability=True))
pipeline.fit(X_train, y_train)

# --- Evaluate on validation set ---
predicted_classes = pipeline.predict(X_val)
probs = pipeline.predict_proba(X_val)
positive_probs = probs[:, 1]  # probability of class 1

# Combine both into a display
for i, (pred, prob) in enumerate(zip(predicted_classes, positive_probs)):
    print(f"Sample {i}: Predicted = {pred}, Prob(class 1) = {prob:.4f}")

# y_pred = svm.predict(X_val)
acc = accuracy_score(y_val, predicted_classes)
print(f"Validation Accuracy: {acc:.4f}")

# print("Classification Report:")
# print(classification_report(y_val, y_pred))

# print(y_pred)

Sample 0: Predicted = 0, Prob(class 1) = 0.0538
Sample 1: Predicted = 0, Prob(class 1) = 0.0000
Sample 2: Predicted = 0, Prob(class 1) = 0.0000
Sample 3: Predicted = 0, Prob(class 1) = 0.0467
Sample 4: Predicted = 0, Prob(class 1) = 0.0000
Sample 5: Predicted = 0, Prob(class 1) = 0.0000
Sample 6: Predicted = 1, Prob(class 1) = 0.9927
Sample 7: Predicted = 0, Prob(class 1) = 0.0000
Sample 8: Predicted = 0, Prob(class 1) = 0.0510
Sample 9: Predicted = 1, Prob(class 1) = 0.9937
Sample 10: Predicted = 0, Prob(class 1) = 0.0578
Sample 11: Predicted = 1, Prob(class 1) = 0.9676
Sample 12: Predicted = 0, Prob(class 1) = 0.0000
Sample 13: Predicted = 0, Prob(class 1) = 0.0568
Sample 14: Predicted = 1, Prob(class 1) = 0.9956
Sample 15: Predicted = 0, Prob(class 1) = 0.0000
Sample 16: Predicted = 0, Prob(class 1) = 0.0642
Sample 17: Predicted = 1, Prob(class 1) = 0.9514
Sample 18: Predicted = 0, Prob(class 1) = 0.0000
Sample 19: Predicted = 1, Prob(class 1) = 1.0000
Sample 20: Predicted = 0, Prob

In [8]:
import joblib

# Save the SVM model
joblib.dump(pipeline, 'models/svm/svm_model.pkl')

['models/svm/svm_model.pkl']

In [9]:
model_path = r"C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\svm\svm_model.pkl"
model_name = model_path.split('\\')[-1]

print(f"Creating model {model_name}...")
svm_loaded = joblib.load(model_path)

print("Model loaded and moved to device")

Creating model svm_model.pkl...
Model loaded and moved to device


In [12]:
# --- Evaluate on validation set ---
predicted_classes = svm_loaded.predict(X_val)
probs = svm_loaded.predict_proba(X_val)
positive_probs = probs[:, 1]  # probability of class 1

# Combine both into a display
for i, (pred, prob) in enumerate(zip(predicted_classes, positive_probs)):
    print(f"Sample {i}: Predicted = {pred}, Prob(class 1) = {prob:.4f}")

# y_pred = svm.predict(X_val)
acc = accuracy_score(y_val, predicted_classes)
print(f"Validation Accuracy: {acc:.4f}")

Sample 0: Predicted = 0, Prob(class 1) = 0.0538
Sample 1: Predicted = 0, Prob(class 1) = 0.0000
Sample 2: Predicted = 0, Prob(class 1) = 0.0000
Sample 3: Predicted = 0, Prob(class 1) = 0.0467
Sample 4: Predicted = 0, Prob(class 1) = 0.0000
Sample 5: Predicted = 0, Prob(class 1) = 0.0000
Sample 6: Predicted = 1, Prob(class 1) = 0.9927
Sample 7: Predicted = 0, Prob(class 1) = 0.0000
Sample 8: Predicted = 0, Prob(class 1) = 0.0510
Sample 9: Predicted = 1, Prob(class 1) = 0.9937
Sample 10: Predicted = 0, Prob(class 1) = 0.0578
Sample 11: Predicted = 1, Prob(class 1) = 0.9676
Sample 12: Predicted = 0, Prob(class 1) = 0.0000
Sample 13: Predicted = 0, Prob(class 1) = 0.0568
Sample 14: Predicted = 1, Prob(class 1) = 0.9956
Sample 15: Predicted = 0, Prob(class 1) = 0.0000
Sample 16: Predicted = 0, Prob(class 1) = 0.0642
Sample 17: Predicted = 1, Prob(class 1) = 0.9514
Sample 18: Predicted = 0, Prob(class 1) = 0.0000
Sample 19: Predicted = 1, Prob(class 1) = 1.0000
Sample 20: Predicted = 0, Prob